# Caso precio huevo rojo extra
Profesores:  Jorge Romero

<img src="https://estaticos.muyinteresante.es/media/cache/1140x_thumb/uploads/images/article/5536592a70a1ae8d775df122/huevos_1.jpg" alt="" width="25%" height="10%">

*   Son necesarios para planeación a futuro.
*   Casi siempre se equivocan.
*   Debemos calcular su nivel de error.

<img src="https://media0.giphy.com/media/3o6Mb8HJCu3g3qXIGY/giphy.gif?cid=ecf05e47uvgtjfrmyvthcbarh60yq5w9uu3oq0t0g91c2eh1&rid=giphy.gif" alt="" width="40%" height="10%">


## Comencemos con un conjunto de datos de precios historicos de huevos:
<img src="https://fenavi.org/wp-content/uploads/2018/02/logofenavi.png" alt="" width="15%" height="15%">

https://drive.google.com/file/d/1zfk1BmJ8epeVDILeir4OfpXW8e_kpITB/view?usp=sharing

Probemos importar desde internet los históricos de huevo rojo extra en colombia precio mayorista:

```python
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
URL = 'https://drive.google.com/file/d/1aE6xob2J727VqdzEMTKYZs2dzjBAqZxq/view?usp=sharing'
path = 'https://drive.google.com/uc?export=download&id='+URL.split('/')[-2]
df = pd.read_csv(path, ',')
df.head()
``` 
## Copie a continuacion:

Ahora Grafiquemos los precios:

<img src="https://media1.giphy.com/media/3o6MbaIqtYEHYWtJ5u/giphy.gif?cid=ecf05e47eazvdzilnau4udei9pe1sdckeonahbkkfxv7iqw7&rid=giphy.gif" alt="" width="50%" height="25%">


```python
# Imprime la serie de tiempo
f, ax = plt.subplots(1,1)
ax.plot(df['y'])
# Agrega titulo
ax.set_title('Casos diarios')
# Etiquetas ejes
ax.tick_params(axis = 'x', rotation = 45)
plt.xlabel('dias')
plt.ylabel('Casos')
# Muestra grafica
plt.show()
plt.close()
```

Pruebe debajo!!!:

___________________________________

# Pronosticos con FB- Prophet

Prophet es un procedimiento para pronosticar datos de series temporales basados en un modelo aditivo en el que las tendencias no lineales se ajustan a la estacionalidad anual, semanal y diaria, más los efectos de vacaciones. Funciona mejor con series de tiempo que tienen fuertes efectos estacionales y varias temporadas de datos históricos. Prophet es robusto ante los datos faltantes y los cambios en la tendencia, y generalmente maneja bien los valores atípicos.

## **Paso 1:** Funcion para calcular el tiempo
```python
from fbprophet import Prophet
import datetime
from datetime import datetime

def days_between(d1, d2):
    d1 = datetime.strptime(d1, "%Y-%m-%d")
    d2 = datetime.strptime(d2, "%Y-%m-%d")
    return abs((d2 - d1).days + 1)

```
_______________________________________



## **Paso 2:** Agregar condiciones
```python
date_column = 'dt'
metric_column = 'ts'
start_training_date = '2017-01-01'
end_training_date = '2020-01-01'
start_forecasting_date = '2020-02-01'
end_forecasting_date = '2020-09-01'
year_to_estimate = '2020'
future_num_points = days_between(start_forecasting_date, end_forecasting_date)
cap = None 
growth = 'linear'
n_changepoints = 25 
changepoint_prior_scale = 0.05 
changepoints = None 
holidays_prior_scale = 10 
interval_width = 0.8 
mcmc_samples = 0
holidays = None
daily_seasonality = False
yearly_seasonality = True

  ```
_________________________________

## **Paso 3:** Crear funcion de pronostico
```python
URL = 'https://drive.google.com/file/d/1aE6xob2J727VqdzEMTKYZs2dzjBAqZxq/view?usp=sharing'
path = 'https://drive.google.com/uc?export=download&id='+URL.split('/')[-2]
df = pd.read_csv(path, ',')
df_prophet=df
def create_daily_forecast(df,
#                           cap,
                          holidays,
                          growth,
                          n_changepoints = 25,
                          changepoint_prior_scale = 0.05,
                          changepoints = None,
                          holidays_prior_scale = 10,
                          interval_width = 0.8,
                          mcmc_samples = 1,
                          future_num_points = 10, 
                          daily_seasonality = False,
                          yearly_seasonality = True):
  """
  Crear el pronostico
  """
  df_ = df.copy()
  m = Prophet(growth = growth,
              n_changepoints = n_changepoints,
              changepoint_prior_scale = changepoint_prior_scale,
              changepoints = changepoints,
              holidays = holidays,
              holidays_prior_scale = holidays_prior_scale,
              interval_width = interval_width,
              mcmc_samples = mcmc_samples, 
              daily_seasonality = daily_seasonality,
              yearly_seasonality= yearly_seasonality)
  m.fit(df_)
  future = m.make_future_dataframe(5,freq='MS')
  fcst = m.predict(future)
  m.plot(fcst);
  m.plot_components(fcst)
 
  return fcst
  ```
  _____________________________________

## Paso 4 Correr el pronostico
```python
fcst = create_daily_forecast(df_prophet,
                             holidays,
                             growth,
                             n_changepoints,
                             changepoint_prior_scale,
                             changepoints, 
                             holidays_prior_scale,
                             interval_width,
                             mcmc_samples,
                             future_num_points, 
                             daily_seasonality,
                             yearly_seasonality)
```
______________________________

## Finalmente calculemos el error estadistico:
```python
def calculate_mape(y_true, y_pred):
    """ Calculate mean absolute percentage error (MAPE)"""
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def calculate_mpe(y_true, y_pred):
    """ Calculate mean percentage error (MPE)"""
    return np.mean((y_true - y_pred) / y_true) * 100

def calculate_mae(y_true, y_pred):
    """ Calculate mean absolute error (MAE)"""
    return np.mean(np.abs(y_true - y_pred)) * 100

def calculate_rmse(y_true, y_pred):
    """ Calculate root mean square error (RMSE)"""
    return np.sqrt(np.mean((y_true - y_pred)**2))

def print_error_metrics(y_true, y_pred):
    print('MAPE: %f'%calculate_mape(y_true, y_pred))
    print('MPE: %f'%calculate_mpe(y_true, y_pred))
    print('MAE: %f'%calculate_mae(y_true, y_pred))
    print('RMSE: %f'%calculate_rmse(y_true, y_pred))
    return
    
    print_error_metrics(y_true = df_prophet['y'], y_pred = fcst['yhat'])
```

In [25]:
def calculate_mape(y_true, y_pred):
    """ Calculate mean absolute percentage error (MAPE)"""
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
 
def calculate_mpe(y_true, y_pred):
    """ Calculate mean percentage error (MPE)"""
    return np.mean((y_true - y_pred) / y_true) * 100
 
def calculate_mae(y_true, y_pred):
    """ Calculate mean absolute error (MAE)"""
    return np.mean(np.abs(y_true - y_pred)) * 100
 
def calculate_rmse(y_true, y_pred):
    """ Calculate root mean square error (RMSE)"""
    return np.sqrt(np.mean((y_true - y_pred)**2))
 
def print_error_metrics(y_true, y_pred):
    print('MAPE: %f'%calculate_mape(y_true, y_pred))
    print('MPE: %f'%calculate_mpe(y_true, y_pred))
    print('MAE: %f'%calculate_mae(y_true, y_pred))
    print('RMSE: %f'%calculate_rmse(y_true, y_pred))
    return
 
    print_error_metrics(y_true = df_prophet['y'], y_pred = fcst['yhat'])

In [20]:
 print_error_metrics(y_true = df_prophet['y'], y_pred = fcst['yhat'])

MAPE: 0.995160
MPE: -0.019502
MAE: 324.940726
RMSE: 4.010057
